# Linear Regression - Predicting the stock market

## Cleaning

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
from IPython.display import display

In [2]:
df = pd.read_csv("sphist.csv")
df.head(3)

,Date,Open,High,Low,Close,Volume,Adj Close
0,2015-12-07,2090.419922,2090.419922,2066.780029,2077.070068,4.043820e+09,2077.070068
1,2015-12-04,2051.239990,2093.840088,2051.239990,2091.689941,4.214910e+09,2091.689941
2,2015-12-03,2080.709961,2085.000000,2042.349976,2049.620117,4.306490e+09,2049.620117


In [3]:
df['Date']= pd.to_datetime(df['Date']) # Usefull for later df_dates = df[df["Date"] > datetime(year=2015, month=4, day=1)]
df = df.sort_values(by = 'Date')
df = df.reset_index(drop = True)
print(df.shape)
df.head(3)

(16590, 7)


,Date,Open,High,Low,Close,Volume,Adj Close
0,1950-01-03,16.66,16.66,16.66,16.66,1260000.0,16.66
1,1950-01-04,16.85,16.85,16.85,16.85,1890000.0,16.85
2,1950-01-05,16.93,16.93,16.93,16.93,2550000.0,16.93


## Feature engineering

In [4]:
# Creating indicators such as average price last 5 days
# The average Open from the past 5 days.
df['Avg Open 5d'] = df['Open'].rolling(window = 5).mean()
df['Avg Open 5d'] = df['Avg Open 5d'].shift(periods=1)
# The average High from the past 5 days.
df['Avg High 5d'] = df['High'].rolling(window = 5).mean()
df['Avg High 5d'] = df['Avg High 5d'].shift(periods=1)
# The average Low from the past 5 days.
df['Avg Low 5d'] = df['Low'].rolling(window = 5).mean()
df['Avg Low 5d'] = df['Avg Low 5d'].shift(periods=1)
# The average Volume from the past 5 days.
df['Avg Vol 5d'] = df['Volume'].rolling(window = 5).mean()
df['Avg Vol 5d'] = df['Avg Vol 5d'].shift(periods=1)
# Year
df['Year'] = df['Date'].apply(lambda x: x.year)

df.head(7)

,Date,Open,High,Low,Close,Volume,Adj Close,Avg Open 5d,Avg High 5d,Avg Low 5d,Avg Vol 5d,Year
0,1950-01-03,16.660000,16.660000,16.660000,16.660000,1260000.0,16.660000,NaN,NaN,NaN,NaN,1950
1,1950-01-04,16.850000,16.850000,16.850000,16.850000,1890000.0,16.850000,NaN,NaN,NaN,NaN,1950
2,1950-01-05,16.930000,16.930000,16.930000,16.930000,2550000.0,16.930000,NaN,NaN,NaN,NaN,1950
3,1950-01-06,16.980000,16.980000,16.980000,16.980000,2010000.0,16.980000,NaN,NaN,NaN,NaN,1950
4,1950-01-09,17.080000,17.080000,17.080000,17.080000,2520000.0,17.080000,NaN,NaN,NaN,NaN,1950
5,1950-01-10,17.030001,17.030001,17.030001,17.030001,2160000.0,17.030001,16.900,16.900,16.900,2046000.0,1950
6,1950-01-11,17.090000,17.090000,17.090000,17.090000,2630000.0,17.090000,16.974,16.974,16.974,2226000.0,1950


In [5]:
# Removing NaN (the first 5 rows from window 5 method)
df = df[df["Date"] >= datetime(year=1950, month=1, day=10)]
# Removing other NaN
df = df.dropna(axis = 0)
print(df.shape)

(16585, 12)


## Applying Linear Regression

In [6]:
# Split
train = df[df["Date"] < datetime(year=2013, month=1, day=1)]
test = df[df["Date"] >= datetime(year=2013, month=1, day=1)]

In [7]:
# Predict
features = ['Avg Open 5d', 'Avg High 5d', 'Avg Low 5d', 'Avg Vol 5d', 'Year']
target = 'Close'
reg = LinearRegression().fit(train[features], train[target])
predictions = reg.predict(test[features])

mean_absolute_error = np.mean(np.absolute(test[target] - predictions))
mse = mean_squared_error(test[target], predictions)
rmse = np.sqrt(mse)

print('mean_absolute_error = ', mean_absolute_error)
print('rmse = ', rmse)

mean_absolute_error =  14.594644448369127
rmse =  19.464426623351724


In [8]:
# Results
results = test.copy()
results['Predicted close price'] = predictions
results['Difference'] = results['Predicted close price'] - results['Close']
results['Accuracy(%)'] = (1-np.abs(results['Difference'] / results['Close']))*100
results = results.sort_values(by= ['Date'] , ascending = False)
results = results.drop(columns =['Open', 'High', 'Low', 'Volume', 'Adj Close', 'Avg Open 5d', 'Avg High 5d', 'Avg Low 5d', 'Avg Vol 5d', 'Year'])
results.head()

,Date,Close,Predicted close price,Difference,Accuracy(%)
16589,2015-12-07,2077.070068,2083.850839,6.780771,99.673542
16588,2015-12-04,2091.689941,2069.229331,-22.460610,98.926198
16587,2015-12-03,2049.620117,2085.902406,36.282289,98.229804
16586,2015-12-02,2079.510010,2090.749987,11.239977,99.459489
16585,2015-12-01,2102.629883,2081.031411,-21.598472,98.972788


## Other comments

#### Other possible indicators we could have used

In [9]:
# The standard deviation of the price over the past 5 days.
df['std_dev_5d'] = df['Close'].rolling(window = 5).std()
df['std_dev_5d'] = df['std_dev_5d'].shift(periods=1)
# The average price from the past 5 days.
df['avg_5d'] = df['Close'].rolling(window = 5).mean()
df['avg_5d'] = df['avg_5d'].shift(periods=1)
# The average price from the past 365 days.
df['avg_365d'] = df['Close'].rolling(window = 365).mean()
df['avg_365d'] = df['avg_365d'].shift(periods=1)
# The standard deviation of the price over the past 5 days.
df['std_dev_365d'] = df['Close'].rolling(window = 365).std()
df['std_dev_365d'] = df['std_dev_365d'].shift(periods=1)
# Ratio : average price past 5 days / average price past 365 days.
df['5/365_avg'] = df['avg_5d'] / df['avg_365d']
# Ratio : standard deviation price past 5 days / standard deviation price past 365 days.
df['5/365_std_dev'] = df['std_dev_5d'] / df['std_dev_365d']

#### More indicators that dataquest suggests to use:

In [10]:
# The average volume over the past five days.
# The average volume over the past year.
# The ratio between the average volume for the past five days, and the average volume for the past year.
# The standard deviation of the average volume over the past five days.
# The standard deviation of the average volume over the past year.
# The ratio between the standard deviation of the average volume for the past five days, and the standard deviation of the average volume for the past year.
# The year component of the date.
# The ratio between the lowest price in the past year and the current price.
# The ratio between the highest price in the past year and the current price.
# The month component of the date.
# The day of week.
# The day component of the date.
# The number of holidays in the prior month.

#### Dataquest ideas for making a strong model

There's a lot of things we can do on the indicator side. We can also make significant structural improvements to the algorithm and pull in data from other sources.

Accuracy would improve greatly by making predictions only one day ahead. For example, train a model using data from 1951-01-03 to 2013-01-02, make predictions for 2013-01-03, and then train another model using data from 1951-01-03 to 2013-01-03, make predictions for 2013-01-04, and so on. This more closely simulates what you'd do if you were trading using the algorithm.

You can also improve the algorithm used significantly. Try other techniques, like a random forest, and see if they perform better.

You can also incorporate outside data, such as the weather in New York City (where most trading happens) the day before and the amount of Twitter activity around certain stocks.

You can also make the system real-time by writing an automated script to download the latest data when the market closes and make predictions for the next day.

Finally, you can make the system "higher-resolution". You're currently making daily predictions, but you could make hourly, minute-by-minute, or second-by-second predictions. This requires obtaining more data, though. You could also make predictions for individual stocks instead of the S&P500.